# Paso 2: Descarga Masiva de Datos

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Definimos el portafolio
tickers = ['AAPL', 'TSLA', 'MSFT', 'GOOGL']

# 2. Descargamos 2 años de datos ajustados (Cierre Ajustado)
print("📥 Descargando datos del mercado...")
datos = yf.download(tickers, period="2y")['Close']

# 3. Limpieza rápida (por si acaso)
datos = datos.dropna()

# 4. Vemos las primeras filas
datos.head()

# 🧠 Paso 3: El secreto de los Economistas (Retornos Logarítmicos)

In [ ]:
# Calcular retornos diarios (cambio porcentual)
retornos = datos.pct_change().dropna()

# Graficar para ver el "ruido" del mercado
plt.figure(figsize=(12,6))
for accion in retornos.columns:
    plt.plot(retornos.index, retornos[accion], label=accion, alpha=0.7)

plt.title("Volatilidad Diaria: ¿Quién se mueve más brusco?")
plt.legend()
plt.ylabel("Variación Diaria (%)")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Calcular la Matriz de Correlación
correlacion = retornos.corr()

# Mostrarla con colores (Estilo semáforo)
# Rojo fuerte = Se mueven muy igual (Peligro de concentración)
# Azul/Clarito = Se mueven diferente (Buena diversificación)
correlacion.style.background_gradient(cmap='coolwarm')

¡Lectura perfecta, Axel! 🎯 Has interpretado la matriz como un verdadero gestor de fondos.

Tus datos cuentan una historia económica clara:

El Riesgo Sistémico (Google vs Microsoft - 0.48): Es lógico que tengan la correlación más alta. Ambas son gigantes de "Cloud & IA". Si sale una ley que regule la Inteligencia Artificial, ambas acciones caerán juntas. Aquí hay poca protección.

La Oportunidad de Cobertura (Microsoft vs Tesla - 0.39): Esta es tu joya. Tesla es "Consumo Discrecional/Automotriz" y Microsoft es "Software Empresarial". Sus ciclos económicos son distintos. Combinarlas reduce tu riesgo.

Y si miras el gráfico de líneas, esa línea roja que parece un electrocardiograma de infarto es Tesla. Es muy volátil, pero esa volatilidad es el precio a pagar por sus altos retornos potenciales.

# 🎲 Módulo 5: La Simulación de Montecarlo

In [ ]:
# Misión: Simulación de Montecarlo (5000 futuros posibles)

num_portafolios = 5000
resultados = [] # Aquí guardaremos los datos de cada simulación

print("🎰 Iniciando simulación de 5,000 combinaciones...")

for i in range(num_portafolios):
    # 1. Generar pesos aleatorios (que sumen 100%)
    pesos = np.random.random(len(tickers))
    pesos /= np.sum(pesos)
    
    # 2. Calcular Retorno y Volatilidad esperada de esa combinación
    # (Matemática vectorial para escalar a 1 año de trading = 252 días)
    retorno_port = np.sum(retornos.mean() * pesos) * 252
    volatilidad_port = np.sqrt(np.dot(pesos.T, np.dot(retornos.cov() * 252, pesos)))
    
    # 3. Calcular Ratio de Sharpe (Rendimiento / Riesgo)
    sharpe = retorno_port / volatilidad_port
    
    # 4. Guardar todo
    resultados.append([retorno_port, volatilidad_port, sharpe, pesos])

# Convertir a DataFrame para analizar
df_simulacion = pd.DataFrame(resultados, columns=['Retorno', 'Volatilidad', 'Sharpe', 'Pesos'])

print("✅ ¡Simulación completada!")
df_simulacion.head()

# La Frontera Eficiente

In [ ]:
# Graficar la Frontera Eficiente
plt.figure(figsize=(10, 6))
plt.scatter(df_simulacion['Volatilidad'], df_simulacion['Retorno'], 
            c=df_simulacion['Sharpe'], cmap='viridis', marker='o', s=10, alpha=0.5)

plt.colorbar(label='Sharpe Ratio (Más amarillo = Mejor)')
plt.xlabel('Riesgo (Volatilidad)')
plt.ylabel('Retorno Esperado (Anual)')
plt.title('Frontera Eficiente: Buscando el Portafolio Perfecto')
plt.show()

In [ ]:
# Encontrar el portafolio con el MÁXIMO Ratio de Sharpe
indice_ganador = df_simulacion['Sharpe'].idxmax()
portafolio_optimo = df_simulacion.loc[indice_ganador]

print("🏆 EL PORTAFOLIO GANADOR ES:")
print("-" * 30)
print(f"Retorno Esperado (Anual): {portafolio_optimo['Retorno']*100:.2f}%")
print(f"Riesgo (Volatilidad):     {portafolio_optimo['Volatilidad']*100:.2f}%")
print(f"Ratio Sharpe:             {portafolio_optimo['Sharpe']:.2f}")
print("-" * 30)
print("ASIGNACIÓN DE ACTIVOS (Tu Receta):")

# Sacamos los pesos y los mostramos bonitos
pesos_optimos = portafolio_optimo['Pesos']
for i, empresa in enumerate(tickers):
    print(f"{empresa}: {pesos_optimos[i]*100:.2f}%")

In [ ]:
# Graficar la Torta de Inversión
plt.figure(figsize=(7, 7))
plt.pie(pesos_optimos, labels=tickers, autopct='%1.1f%%', startangle=140, colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
plt.title('Tu Portafolio Óptimo (Maximiza Sharpe)')
plt.show()

# Módulo 6: El Guardián Silencioso (Automatización)

In [ ]:
import time
from datetime import datetime

# Definimos nuestra acción favorita del portafolio
accion_vigilada = "TSLA"
precio_objetivo = 1000 # Ponemos un precio alto para que "crea" que está barata y nos avise

print(f"🤖 INICIANDO ROBOT VIGILANTE PARA {accion_vigilada}...")
print("Presiona el botón de 'Stop' (cuadrado negro arriba) para apagarme.\n")

# El Bucle Infinito (El corazón del Bot)
while True:
    # 1. Obtener precio en tiempo real (usamos la última data descargada como simulación)
    # En un bot real, aquí haríamos yf.download() otra vez.
    # Para este ejemplo, simulamos un precio aleatorio basado en el último cierre
    ultimo_precio = datos[accion_vigilada].iloc[-1] 
    
    # Simulamos una pequeña fluctuación del mercado en vivo
    precio_simulado = ultimo_precio * np.random.uniform(0.98, 1.02)
    
    hora_actual = datetime.now().strftime("%H:%M:%S")
    
    print(f"[{hora_actual}] {accion_vigilada} está a ${precio_simulado:.2f}...")

    # 2. La Decisión (El Cerebro)
    if precio_simulado < precio_objetivo:
        print("    🚨 ¡ALERTA DE OPORTUNIDAD! 🚨")
        print(f"    >>> {accion_vigilada} está barata (Bajo ${precio_objetivo}). ¡COMPRAR YA!")
        print("    ------------------------------------------------")
    
    # 3. Esperar antes de volver a chequear (para no bloquear la PC)
    time.sleep(5) # Espera 5 segundos

# Paso 1: Preparar el Experimento

In [ ]:
from sklearn.linear_model import LinearRegression

# 1. Descargamos datos: Tesla vs El Mercado (SPY)
print("📥 Descargando datos para el duelo...")
data_ml = yf.download(['TSLA', 'SPY'], period="2y")['Close']

# 2. Calculamos los retornos diarios (La gasolina del modelo)
retornos_ml = data_ml.pct_change().dropna()

# 3. Separamos X (El Mercado) e Y (Tesla)
# El modelo necesita arrays de 2 dimensiones, por eso usamos .values.reshape(-1, 1)
X = retornos_ml['SPY'].values.reshape(-1, 1) # Variable Independiente
Y = retornos_ml['TSLA'].values.reshape(-1, 1) # Variable Dependiente (Lo que queremos explicar)

print("✅ Datos listos para entrenar el modelo.")

# Paso 2: Entrenar el Cerebro Artificial

In [ ]:
# 1. Crear el modelo
modelo = LinearRegression()

# 2. Entrenar el modelo (Aprender de la historia)
modelo.fit(X, Y)

# 3. Extraer los coeficientes (La "Magia" Económica)
beta = modelo.coef_[0][0]
alpha = modelo.intercept_[0]

print(f"📈 RESULTADOS DEL MODELO ECONOMÉTRICO:")
print(f"---------------------------------------")
print(f"🦁 Beta de Tesla (Riesgo): {beta:.4f}")
print(f"👻 Alpha (Retorno extra):  {alpha:.4f}")
print(f"---------------------------------------")

if beta > 1:
    print("CONCLUSIÓN: Tesla es AGRESIVA (Se mueve más que el mercado).")
elif beta < 1:
    print("CONCLUSIÓN: Tesla es DEFENSIVA (Se mueve menos que el mercado).")

# Paso 3: Visualizar la Regresión

In [ ]:
# Graficar la dispersión y la línea de regresión
plt.figure(figsize=(10, 6))

# Puntos reales (Scatter)
plt.scatter(X, Y, alpha=0.3, color='blue', label='Días Reales de Trading')

# Línea de Predicción (La Regresión)
plt.plot(X, modelo.predict(X), color='red', linewidth=2, label=f'Línea de Tendencia (Beta={beta:.2f})')

plt.title('Modelo CAPM: Sensibilidad de Tesla vs S&P 500')
plt.xlabel('Retornos del Mercado (S&P 500)')
plt.ylabel('Retornos de Tesla')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.show()

In [ ]:
!pip install pandas-datareader

In [ ]:
import pandas_datareader.data as web
import datetime

# 1. Definimos el periodo (5 años para ver ciclos económicos)
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime.now()

print("🏛️ Conectando con la Reserva Federal (FRED API)...")

# 2. Pedimos el dato 'DGS10' (10-Year Treasury Constant Maturity Rate)
tasa_interes = web.DataReader('DGS10', 'fred', start, end)

# 3. Pedimos los datos de Tesla (Yahoo Finance)
print("📈 Conectando con el Mercado de Valores...")
tesla = yf.download('TSLA', start=start, end=end)['Close']

# 4. Unimos todo en un solo DataFrame (Limpiamos datos vacíos)
df_macro = pd.DataFrame()
df_macro['Tasa_Fed'] = tasa_interes
df_macro['Tesla_Precio'] = tesla
df_macro = df_macro.dropna()

print("✅ Datos Macro-Financieros Listos:")
df_macro.head()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:red'
ax1.set_xlabel('Fecha')
ax1.set_ylabel('Tasa del Tesoro 10 Años (%)', color=color)
ax1.plot(df_macro.index, df_macro['Tasa_Fed'], color=color, linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)

# Creamos el segundo eje (gemelo)
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('Precio de Tesla (USD)', color=color)  
ax2.plot(df_macro.index, df_macro['Tesla_Precio'], color=color, linewidth=2, alpha=0.6)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Impacto de la Política Monetaria en Tech Stocks (Macro-Finance)')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

# 1. Preparamos el DataFrame con el formato que exige Facebook Prophet
# Usamos los datos de Tesla que bajamos en el paso anterior (df_macro)
df_forecast = df_macro[['Tesla_Precio']].reset_index()
df_forecast.columns = ['ds', 'y'] # Renombramos las columnas obligatoriamente

# 2. Creamos el Cerebro de la IA
# daily_seasonality=True ayuda a captar patrones de corto plazo
m = Prophet(daily_seasonality=True)

# 3. Entrenamos (Le enseñamos toda la historia de Tesla)
print("🧠 Entrenando la IA con los ciclos históricos...")
m.fit(df_forecast)

print("✅ Modelo entrenado. Listo para ver el futuro.")

In [ ]:
# 1. Crear un dataframe con fechas futuras (365 días adelante)
future = m.make_future_dataframe(periods=365)

# 2. Predecir
print("🔮 Calculando predicciones para el próximo año...")
forecast = m.predict(future)

# 3. Ver los números (Las últimas 5 filas son del futuro)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# Graficar la predicción
fig1 = m.plot(forecast)

plt.title('Predicción Econométrica de Tesla (Prophet AI)')
plt.xlabel('Fecha')
plt.ylabel('Precio Estimado')
plt.show()

# Módulo 10: Gestión de Riesgo Institucional (VaR)

In [ ]:
# 1. Calculamos los retornos diarios de Tesla (si no los teníamos ya frescos)
retornos_tesla = df_macro['Tesla_Precio'].pct_change().dropna()

# 2. Elegimos el Nivel de Confianza (99% es estándar en bancos)
nivel_confianza = 0.99

# 3. Calculamos el VaR (Value at Risk)
# Buscamos el "cuantil" peor del historial.
# Si pedimos confianza 99%, buscamos el 1% de los peores días.
var_99 = np.percentile(retornos_tesla, 100 * (1 - nivel_confianza))

print(f"💀 VALUE AT RISK (VaR 99%): {var_99*100:.2f}%")

# 4. Traducirlo a Dinero (Ejemplo: Si invertimos $10,000 USD)
inversion = 10000
perdida_maxima = inversion * var_99

print(f"💸 Interpretación: Con un 99% de seguridad, NO perderemos más de ${abs(perdida_maxima):.2f} USD en un día.")
print(f"   (O dicho de otra forma: Solo 1 de cada 100 días será peor que esto).")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns # Librería para gráficos estadísticos bonitos

plt.figure(figsize=(10, 6))

# 1. Histograma de los retornos (La Campana)
sns.histplot(retornos_tesla, bins=50, kde=True, color='skyblue', label='Días Normales')

# 2. Línea del VaR (El límite del dolor)
plt.axvline(var_99, color='red', linestyle='--', linewidth=3, label=f'VaR 99% ({var_99*100:.2f}%)')

# 3. Decoración
plt.title('Distribución de Riesgo de Tesla: ¿Dónde están los "Cisnes Negros"?')
plt.xlabel('Retorno Diario')
plt.ylabel('Frecuencia')
plt.legend()
plt.show()